# Available Land Finder (City + Road Setback + Not Water)

Goal: Identify land that is:

- Within **city limits**
- At least **max(NUM_LANES * 10 + 6, 30) feet** away from road centerlines  
  (If `NUM_LANES` is missing or null, the rule becomes **30 feet**)
- **Not water**

This notebook is written for **ArcGIS Pro / ArcPy** and is organized for straightforward debugging.


In [42]:
import os
import arcpy
from tkinter import Tk, filedialog

arcpy.env.overwriteOutput = True

print('ArcPy ready.')
print('ArcGIS Pro scratchGDB:', arcpy.env.scratchGDB)


ArcPy ready.
ArcGIS Pro scratchGDB: C:\Users\Ryan Sibley\Desktop\MURP\Fall 2025\FinalProjects\FinalProjectsFinal\scratch.gdb


## Helper functions

These helpers keep the workflow readable and reduce copy/paste bugs.


In [43]:
def _safe_name(text, max_len=60):
    text = str(text)
    out = []
    for ch in text:
        out.append(ch if (ch.isalnum() or ch in ("_", "-")) else "_")
    s = "".join(out).strip("_")
    if not s:
        s = "City"
    return s[:max_len]

def _delete_if_exists(path):
    try:
        if path and arcpy.Exists(path):
            arcpy.management.Delete(path)
    except Exception:
        pass

def _pick_file(title, filetypes):
    root = Tk()
    root.withdraw()
    path = filedialog.askopenfilename(title=title, filetypes=filetypes)
    if not path:
        raise RuntimeError("No file selected.")
    return path

def _pick_folder(title):
    root = Tk()
    root.withdraw()
    folder = filedialog.askdirectory(title=title)
    if not folder:
        raise RuntimeError("No folder selected.")
    return folder

def _get_or_create_map(aprx, map_name):
    for m in aprx.listMaps():
        if m.name == map_name:
            return m
    return aprx.createMap(map_name)


## Step 1 — Select input datasets

You will be prompted to select:

1. A **city boundaries** polygon layer (shapefile or feature class)
2. The **name field** in that layer that contains city names (you will type it)
3. A **roads centerline** line layer
4. A **water** polygon layer (lakes/rivers/wetlands). This is erased from the output.

Notes:
- Roads should include a `NUM_LANES` field if you want lane-based setbacks. If it does not exist, the setback becomes 30 feet.
- City boundary must be polygons.


In [44]:
# 1) Choose datasets
boundary_fc = _pick_file(
    "Select CITY BOUNDARY polygons (shp or feature class)",
    [("Shapefile", "*.shp"), ("All files", "*.*")]
)

roads_fc = _pick_file(
    "Select ROAD CENTERLINES (shp or feature class)",
    [("Shapefile", "*.shp"), ("All files", "*.*")]
)

water_fc = _pick_file(
    "Select WATER polygons (shp or feature class)",
    [("Shapefile", "*.shp"), ("All files", "*.*")]
)

parcels_fc = _pick_file(
    "Select CITY PARCEL polygons to exclude (shp or feature class)",
    [("Shapefile", "*.shp"), ("All files", "*.*")]
)

print("Boundary:", boundary_fc)
print("Roads:   ", roads_fc)
print("Water:   ", water_fc)
print("Parcels: ", parcels_fc)

# 2) Pick city name field
print("\nBoundary fields:")
for f in arcpy.ListFields(boundary_fc):
    print(" -", f.name)

name_field = input("\nType the CITY NAME field (exactly as shown above): ").strip()
if not name_field:
    raise RuntimeError("City name field is required.")

# 3) List unique cities and pick one
cities = sorted({row[0] for row in arcpy.da.SearchCursor(boundary_fc, [name_field]) if row[0] not in (None, "")})
print("\nCities found (first 50 shown):")
for c in cities[:50]:
    print(" -", c)
if len(cities) > 50:
    print(f"... ({len(cities)} total)")

city_name = input("\nType a city name exactly as listed above: ").strip()
if city_name not in cities:
    raise RuntimeError("City name not found in boundary layer. Check spelling and field selection.")

print("Selected city:", city_name)

Boundary: C:/Users/Ryan Sibley/Desktop/GIS/city_township_unorg.shp
Roads:    C:/Users/Ryan Sibley/Desktop/GIS/RoadCenterline.shp
Water:    C:/Users/Ryan Sibley/Desktop/GIS/LakesAndRivers.shp
Parcels:  C:/Users/Ryan Sibley/Desktop/GIS/County_Parcels.shp

Boundary fields:
 - FID
 - Shape
 - GNIS_FEATU
 - FEATURE_NA
 - CTU_CLASS
 - COUNTY_GNI
 - COUNTY_COD
 - COUNTY_NAM
 - POPULATION
 - SHAPE_Leng
 - SHAPE_Area



Type the CITY NAME field (exactly as shown above):  FEATURE_NA



Cities found (first 50 shown):
 -  
 - Aastad
 - Acoma
 - Acton
 - Ada
 - Adams
 - Adrian
 - Aetna
 - Afton
 - Agassiz
 - Agder
 - Agram
 - Aitkin
 - Akeley
 - Akron
 - Alango
 - Alaska
 - Alba
 - Albany
 - Albert Lea
 - Alberta
 - Albertville
 - Albin
 - Albion
 - Alborn
 - Alden
 - Aldrich
 - Alexandria
 - Alfsborg
 - Alliance
 - Alma
 - Almond
 - Alpha
 - Alta Vista
 - Alton
 - Altona
 - Altura
 - Alvarado
 - Alvwood
 - Amador
 - Amboy
 - Amherst
 - Amiret
 - Amo
 - Amor
 - Andover
 - Andrea
 - Angora
 - Angus
 - Ann
... (2250 total)



Type a city name exactly as listed above:  Amo


Selected city: Amo


## Step 2 — Choose output folder

The final output saved to disk will be:

- `{CityName}_AvailableLand.shp`

A folder named `generatedmaps` will be created in your selected output folder (if missing).


In [45]:
output_root = _pick_folder("Select a folder to store outputs (a generatedmaps folder will be created)")
generatedmaps_folder = os.path.join(output_root, "generatedmaps")
os.makedirs(generatedmaps_folder, exist_ok=True)

final_name = f"{_safe_name(city_name)}_AvailableLand.shp"
final_output = os.path.join(generatedmaps_folder, final_name)

# If it already exists, create a unique name
if os.path.exists(final_output):
    final_output = arcpy.CreateUniqueName(final_name, generatedmaps_folder)

print("Output folder:", generatedmaps_folder)
print("Final output: ", final_output)


Output folder: C:/Users/Ryan Sibley/Desktop/GIS/metadata\generatedmaps
Final output:  C:/Users/Ryan Sibley/Desktop/GIS/metadata\generatedmaps\Amo_AvailableLand.shp


## Step 3 — Run the workflow

This produces the final available land:

1. Select city polygon from boundaries
2. Clip roads + water to city (for speed and cleaner outputs)
3. Create a road buffer using: `max(NUM_LANES * 10 + 6, 30)` feet  
4. Erase road buffer from city
5. Erase water from result
6. Save final shapefile and add to a map called `generatedcities`

Intermediates are written to the project scratch geodatabase and deleted at the end.


In [46]:
# -----------------------------
# Set up scratch workspace
# -----------------------------
scratch_gdb = arcpy.env.scratchGDB
if not scratch_gdb or not arcpy.Exists(scratch_gdb):
    raise RuntimeError("scratchGDB not available. Run inside an ArcGIS Pro project.")

tag = _safe_name(city_name)

city_fc = os.path.join(scratch_gdb, f"{tag}_city")
roads_clip = os.path.join(scratch_gdb, f"{tag}_roads_clip")
water_clip = os.path.join(scratch_gdb, f"{tag}_water_clip")
parcels_clip = os.path.join(scratch_gdb, f"{tag}_parcels_clip")
available_no_parcels = os.path.join(scratch_gdb, f"{tag}_available_no_parcels")
road_buffer = os.path.join(scratch_gdb, f"{tag}_road_buffer")
city_minus_roads = os.path.join(scratch_gdb, f"{tag}_city_minus_roads")
available_land = os.path.join(scratch_gdb, f"{tag}_available_land")
available_singlepart = os.path.join(scratch_gdb, f"{tag}_available_singlepart")
available_filtered = os.path.join(scratch_gdb, f"{tag}_available_filtered")

for p in [city_fc, roads_clip, water_clip, parcels_clip, road_buffer, city_minus_roads,
          available_land, available_no_parcels, available_singlepart, available_filtered]:
    _delete_if_exists(p)
print("Existing temp layers deleted.")

# -----------------------------
# 1) Select city polygon
# -----------------------------
print("Building city polygon feature class...")
field_delim = arcpy.AddFieldDelimiters(boundary_fc, name_field)
city_sql = city_name.replace("'", "''")
where = field_delim + " = '" + city_sql + "'"

city_layer = "city_layer"
arcpy.management.MakeFeatureLayer(boundary_fc, city_layer, where)

cnt = int(arcpy.management.GetCount(city_layer)[0])
if cnt == 0:
    raise RuntimeError("City selection returned 0 features. Check city name and name field.")

arcpy.management.CopyFeatures(city_layer, city_fc)

# -----------------------------
# 2) Clip roads and water to city
# -----------------------------
print("Clipping roads and water to city...")
arcpy.analysis.PairwiseClip(roads_fc, city_fc, roads_clip)
arcpy.analysis.PairwiseClip(water_fc, city_fc, water_clip)

if not arcpy.Exists(roads_clip):
    raise RuntimeError("Road clip output not created. Check roads input and geometry.")
if not arcpy.Exists(water_clip):
    raise RuntimeError("Water clip output not created. Check water input and geometry.")

print("Repairing geometry...")
arcpy.management.RepairGeometry(roads_clip)
arcpy.management.RepairGeometry(water_clip)

# -----------------------------
# 3) Road buffer distance rule
#    max(NUM_LANES * 10 + 6, 30)
# -----------------------------
print("Creating road buffer...")
roads_fields = [f.name.upper() for f in arcpy.ListFields(roads_clip)]

if "NUM_LANES" in roads_fields:
    if "BUFFER_FT" not in roads_fields:
        arcpy.management.AddField(roads_clip, "BUFFER_FT", "DOUBLE")

    code_block = (
        "def setback(n):\n"
        "    try:\n"
        "        v = float(n)\n"
        "    except:\n"
        "        return 30.0\n"
        "    d = v * 10.0 + 6.0\n"
        "    return d if d > 30.0 else 30.0\n"
    )

    arcpy.management.CalculateField(
        in_table=roads_clip,
        field="BUFFER_FT",
        expression="setback(!NUM_LANES!)",
        expression_type="PYTHON3",
        code_block=code_block
    )

    arcpy.analysis.PairwiseBuffer(roads_clip, road_buffer, "BUFFER_FT")
    print("Buffer rule: max(NUM_LANES*10+6, 30)")
else:
    arcpy.analysis.PairwiseBuffer(roads_clip, road_buffer, "30 Feet")
    print("NUM_LANES not found. Buffer rule: 30 feet")

if not arcpy.Exists(road_buffer):
    raise RuntimeError("Road buffer not created. Check road geometry and fields.")

# -----------------------------
# 4) Erase road buffer from city
# -----------------------------
print("Erasing road buffer from city...")
arcpy.analysis.PairwiseErase(city_fc, road_buffer, city_minus_roads)
if not arcpy.Exists(city_minus_roads):
    raise RuntimeError("City-minus-roads not created.")

# -----------------------------
# 5) Erase water from result
# -----------------------------
print("Erasing water...")
water_count = int(arcpy.management.GetCount(water_clip)[0])
if water_count > 0:
    arcpy.analysis.PairwiseErase(city_minus_roads, water_clip, available_land)
else:
    arcpy.management.CopyFeatures(city_minus_roads, available_land)

if not arcpy.Exists(available_land):
    raise RuntimeError("Available land output not created.")

# -----------------------------
# 6) Erase TAXABLE parcels from result (Not a Parcel)
#    Keep parcels where TOTAL_TAX = 0 (or equivalent tax field)
# -----------------------------
print("Filtering parcels (tax > 0), clipping, and erasing...")

# Find a usable tax field (different counties name this differently)
parcel_fields = arcpy.ListFields(parcels_fc)
parcel_field_names = [f.name for f in parcel_fields]

candidate_tax_fields = [
    "TOTAL_TAX", "TOTALTAX", "TOT_TAX", "TOTLTAX", "TAX_TOT",
    "NET_TAX", "NETTAX", "TAX_TOTAL", "TAXTOT", "TAXAMT", "TAX_AMOUNT"
]

tax_fld_obj = None
for cand in candidate_tax_fields:
    for f in parcel_fields:
        if f.name.upper() == cand:
            tax_fld_obj = f
            break
    if tax_fld_obj:
        break

if tax_fld_obj is None:
    raise RuntimeError(
        "No tax field found on parcels_fc. Tried: "
        + ", ".join(candidate_tax_fields)
        + "\nAvailable fields:\n"
        + ", ".join(parcel_field_names)
    )

tax_field = arcpy.AddFieldDelimiters(parcels_fc, tax_fld_obj.name)

# Build a safe where clause for TEXT vs numeric tax fields
if tax_fld_obj.type in ("Double", "Single", "Integer", "SmallInteger"):
    parcel_where = f"{tax_field} > 0"
else:
    # TEXT field: treat any non-empty value not equal to '0' as taxable
    parcel_where = f"{tax_field} IS NOT NULL AND TRIM({tax_field}) <> '' AND TRIM({tax_field}) <> '0'"

print(f"Using tax field: {tax_fld_obj.name} ({tax_fld_obj.type})")
print(f"Parcel filter SQL: {parcel_where}")

parcels_layer = "parcels_taxable_lyr"
arcpy.management.MakeFeatureLayer(parcels_fc, parcels_layer, parcel_where)

taxable_count = int(arcpy.management.GetCount(parcels_layer)[0])
print(f"Taxable parcels selected: {taxable_count}")

if taxable_count > 0:
    arcpy.analysis.PairwiseClip(parcels_layer, city_fc, parcels_clip)
    arcpy.analysis.PairwiseErase(available_land, parcels_clip, available_no_parcels)
else:
    print("No taxable parcels found — skipping parcel erase.")
    arcpy.management.CopyFeatures(available_land, available_no_parcels)

if not arcpy.Exists(available_no_parcels):
    raise RuntimeError("Parcel erase output not created. Check parcel input and tax field.")

# -----------------------------
# 6.5) Multipart → Singlepart, then remove smallest 30% by area
# -----------------------------
import math

print("Converting multipart polygons to singlepart...")
arcpy.management.MultipartToSinglepart(available_no_parcels, available_singlepart)

if not arcpy.Exists(available_singlepart):
    raise RuntimeError("MultipartToSinglepart failed to create output.")

area_field = "AREA_AC"
fields_upper = [f.name.upper() for f in arcpy.ListFields(available_singlepart)]
if area_field not in fields_upper:
    arcpy.management.AddField(available_singlepart, area_field, "DOUBLE")

arcpy.management.CalculateGeometryAttributes(
    in_features=available_singlepart,
    geometry_property=[[area_field, "AREA_GEODESIC"]],
    area_unit="ACRES"
)

rows = []
with arcpy.da.SearchCursor(available_singlepart, ["OID@", area_field]) as cur:
    for oid, a in cur:
        rows.append((float(a or 0.0), int(oid)))

n = len(rows)
if n == 0:
    raise RuntimeError("No polygons found after multipart-to-singlepart.")
elif n < 4:
    print(f"Only {n} polygons found; skipping 'remove smallest 30%' step to avoid over-pruning.")
    arcpy.management.CopyFeatures(available_singlepart, available_filtered)
else:
    rows.sort(key=lambda x: x[0])
    remove_n = int(math.floor(n * 0.30))

    if remove_n <= 0:
        print("remove_n computed as 0; skipping removal.")
        arcpy.management.CopyFeatures(available_singlepart, available_filtered)
    else:
        remove_oids = [oid for _, oid in rows[:remove_n]]
        print(f"Removing {remove_n} smallest polygons out of {n} (~30%).")

        lyr = "avail_singlepart_lyr"
        arcpy.management.MakeFeatureLayer(available_singlepart, lyr)

        def _chunks(lst, size=900):
            for i in range(0, len(lst), size):
                yield lst[i:i + size]

        oid_field = arcpy.Describe(available_singlepart).OIDFieldName
        for chunk in _chunks(remove_oids, 900):
            where = f"{oid_field} IN ({','.join(map(str, chunk))})"
            arcpy.management.SelectLayerByAttribute(lyr, "NEW_SELECTION", where)
            arcpy.management.DeleteFeatures(lyr)

        arcpy.management.CopyFeatures(available_singlepart, available_filtered)

if not arcpy.Exists(available_filtered):
    raise RuntimeError("Filtered polygons output not created.")


Existing temp layers deleted.
Building city polygon feature class...
Clipping roads and water to city...
Repairing geometry...
Creating road buffer...
Buffer rule: max(NUM_LANES*10+6, 30)
Erasing road buffer from city...
Erasing water...
Filtering parcels (tax > 0), clipping, and erasing...
Using tax field: TAX_TOT (Double)
Parcel filter SQL: "TAX_TOT" > 0
Taxable parcels selected: 425330
Converting multipart polygons to singlepart...
Only 1 polygons found; skipping 'remove smallest 30%' step to avoid over-pruning.


In [47]:
# -----------------------------
# 6.75) Recreation classification (single-output analysis step)
#   - Remove tiny slivers (< 50 sq ft)
#   - Compute P/A ratio = perimeter_length / area
#   - Garden: intersects a buffer of LOW-SPEED roads (ROUTE_SYS in 05-10)
#   - Trail:  P/A > 2  (applied last; overrides Garden)
#   - Else:   Wildlife/Agro/Carbon
# -----------------------------
print("Classifying recreational use...")

# ---- Settings (simple + visible) ----
MIN_AREA_SQFT = 50.0
GARDEN_ACCESS_FT = 100.0  # "accessible by / shares a border with buffer" 
TRAIL_RATIO_THRESHOLD = .05

rec_field = "REC_USE"
area_f = "A_SQFT"
perim_f = "P_FT"
ratio_f = "P_A_R"

# ---- 1) Ensure required fields exist ----
existing = {f.name.upper() for f in arcpy.ListFields(available_filtered)}
if rec_field.upper() not in existing:
    arcpy.management.AddField(available_filtered, rec_field, "TEXT", field_length=40)
if area_f.upper() not in existing:
    arcpy.management.AddField(available_filtered, area_f, "DOUBLE")
if perim_f.upper() not in existing:
    arcpy.management.AddField(available_filtered, perim_f, "DOUBLE")
if ratio_f.upper() not in existing:
    arcpy.management.AddField(available_filtered, ratio_f, "DOUBLE")

# ---- 2) Calculate geometry + P/A ratio (US feet) ----
arcpy.management.CalculateGeometryAttributes(
    in_features=available_filtered,
    geometry_property=[[area_f, "AREA"], [perim_f, "PERIMETER_LENGTH"]],
    area_unit="SQUARE_FEET_US",
    length_unit="FEET_US"
)

with arcpy.da.UpdateCursor(available_filtered, [area_f, perim_f, ratio_f, rec_field]) as cur:
    for a, p, r, rec in cur:
        a = float(a) if a else 0.0
        p = float(p) if p else 0.0
        r = (p / a) if a > 0 else None
        cur.updateRow((a, p, r, "Wildlife/Agro/Carbon"))  # default

# ---- 3) Remove tiny polygons (< 50 sq ft) ----
avail_lyr = "avail_lyr"
arcpy.management.MakeFeatureLayer(available_filtered, avail_lyr)

area_delim = arcpy.AddFieldDelimiters(available_filtered, area_f)
arcpy.management.SelectLayerByAttribute(avail_lyr, "NEW_SELECTION", f"{area_delim} < {MIN_AREA_SQFT}")
small_cnt = int(arcpy.management.GetCount(avail_lyr)[0])
if small_cnt > 0:
    print(f"Deleting {small_cnt} polygons < {MIN_AREA_SQFT} sq ft...")
    arcpy.management.DeleteFeatures(avail_lyr)
else:
    print(f"No polygons smaller than {MIN_AREA_SQFT} sq ft.")

# ---- 4) Build LOW-SPEED road buffer for Garden accessibility ----
lowspd_buffer = os.path.join(scratch_gdb, f"{tag}_lowspd_buffer")
_delete_if_exists(lowspd_buffer)

# Find ROUTE_SYS field + create a SQL that works for numeric OR text codes
route_obj = None
for f in arcpy.ListFields(roads_clip):
    if f.name.upper() == "ROUTE_SYS":
        route_obj = f
        break
if route_obj is None:
    raise RuntimeError("Field ROUTE_SYS not found in roads_clip.")

route_field = arcpy.AddFieldDelimiters(roads_clip, route_obj.name)

if route_obj.type in ("Double", "Single", "Integer", "SmallInteger"):
    lowspd_sql = f"{route_field} IN (5,6,7,8,9,10)"
else:
    # Text fields are inconsistent: some datasets store '05', others '5'
    lowspd_sql = f"{route_field} IN ('05','06','07','08','09','10','5','6','7','8','9','10')"

lowspd_lyr = "lowspd_lyr"
arcpy.management.MakeFeatureLayer(roads_clip, lowspd_lyr, lowspd_sql)

lowspd_cnt = int(arcpy.management.GetCount(lowspd_lyr)[0])
print(f"Low-speed roads selected: {lowspd_cnt}")

if lowspd_cnt > 0:
    arcpy.analysis.PairwiseBuffer(lowspd_lyr, lowspd_buffer, f"{GARDEN_ACCESS_FT} Feet")
    buf_cnt = int(arcpy.management.GetCount(lowspd_buffer)[0])
    print(f"Low-speed buffer created: {buf_cnt}")
else:
    lowspd_buffer = None
    print("No low-speed roads found; Gardens will not be assigned.")

# ---- 5) Assign Gardens (accessible plots) ----
# Use INTERSECT with a real accessibility buffer.
# This is intentionally robust when polygons are near but not perfectly touching road centerlines.
if lowspd_buffer and arcpy.Exists(lowspd_buffer):
    garden_lyr = "garden_lyr"
    arcpy.management.MakeFeatureLayer(available_filtered, garden_lyr)

    arcpy.management.SelectLayerByLocation(
        in_layer=garden_lyr,
        overlap_type="INTERSECT",
        select_features=lowspd_buffer,
        selection_type="NEW_SELECTION"
    )

    garden_cnt = int(arcpy.management.GetCount(garden_lyr)[0])
    print(f"Garden candidates selected: {garden_cnt}")

    if garden_cnt > 0:
        arcpy.management.CalculateField(garden_lyr, rec_field, "'Garden'", "PYTHON3")

# ---- 6) Assign Trails (P/A > 2) - applied last to override Garden ----
ratio_delim = arcpy.AddFieldDelimiters(available_filtered, ratio_f)
trail_sql = f"{ratio_delim} > {TRAIL_RATIO_THRESHOLD}"

trail_lyr = "trail_lyr"
arcpy.management.MakeFeatureLayer(available_filtered, trail_lyr, trail_sql)
trail_cnt = int(arcpy.management.GetCount(trail_lyr)[0])
print(f"Trail candidates selected (P/A > {TRAIL_RATIO_THRESHOLD}): {trail_cnt}")

if trail_cnt > 0:
    arcpy.management.CalculateField(trail_lyr, rec_field, "'Trail'", "PYTHON3")

print("Recreation classification complete (REC_USE added, small slivers removed).")


Classifying recreational use...
No polygons smaller than 50.0 sq ft.
Low-speed roads selected: 0
No low-speed roads found; Gardens will not be assigned.
Trail candidates selected (P/A > 0.05): 0
Recreation classification complete (REC_USE added, small slivers removed).


In [48]:
# ---- 7) Add updated available_filtered to map (symbolize manually by REC_USE) ----
aprx = arcpy.mp.ArcGISProject("CURRENT")
generated_map = _get_or_create_map(aprx, "generatedcities")

layer_name = f"{city_name}_AvailableLand"

# Remove prior layer with same name (if it exists)
for lyr in list(generated_map.listLayers()):
    if lyr.name.lower() == layer_name.lower():
        generated_map.removeLayer(lyr)

# Add the updated layer from scratchGDB
new_lyr = generated_map.addDataFromPath(available_filtered)
try:
    new_lyr.name = layer_name
except Exception:
    pass

aprx.save()
print(f"Added layer to map: {layer_name} (symbolize by {rec_field})")


Added layer to map: Amo_AvailableLand (symbolize by REC_USE)


In [49]:

# -----------------------------
# 8) Clean intermediates
# -----------------------------
print("Final File Cleanup...")

# Add every scratch output created in this notebook here
to_delete = [
    city_fc,
    #roads_clip,
    water_clip,
    parcels_clip,
    road_buffer,
    city_minus_roads,
    available_land,
    available_no_parcels,
    available_singlepart,
    available_filtered,
    lowspd_buffer,
    avail_lyr,
    city_layer,
    
    
]

for p in to_delete:
    try:
        _delete_if_exists(p)
    except Exception as e:
        print(f"Cleanup skipped (in use or missing): {p}\n  {e}")

print("Done. Intermediates removed from scratchGDB.")

Final File Cleanup...
Done. Intermediates removed from scratchGDB.


### Debug notes

If something fails, common causes are:

- City selection returns 0 features: wrong `name_field` or city name doesn’t match exactly.
- Clip outputs not created: invalid paths, wrong geometry type, or projection issues.
- Buffer fails: `NUM_LANES` exists but contains text values. The `setback()` function handles most cases.
